In [84]:
import pandas as pd
import altair as alt 
import numpy as np
import datetime as dt
from datetime import timedelta

alt.data_transformers.enable("vegafusion")
alt.themes.enable("fivethirtyeight")

ThemeRegistry.enable('fivethirtyeight')

In [85]:
# Read in survey df 
def read_survey_df(): 
    df = pd.read_csv('/Users/ghazalin/NH_inspections_static/data/NH_SurveyDates_Sep2024.csv')
    df = df.rename(columns={'CMS Certification Number (CCN)':'CCN', 'Survey Date': 'survey_date','Type of Survey':'survey_type', 'Survey Cycle':'survey_cycle', 'Processing Date':'process_date'})
    return df 
survey_df = read_survey_df()

In [86]:
print(survey_df.head(5))

      CCN survey_date           survey_type  survey_cycle process_date
0  015009  2023-03-02      Health Complaint             2   2024-09-01
1  015009  2023-03-02       Health Standard             1   2024-09-01
2  015009  2023-02-15  Fire Safety Standard             1   2024-09-01
3  015009  2019-08-21  Fire Safety Standard             2   2024-09-01
4  015009  2019-08-21       Health Standard             2   2024-09-01


In [87]:
# Read in provider df 
def read_provider_df(): 
    df = pd.read_csv('/Users/ghazalin/NH_inspections_static/data/NH_ProviderInfo_Sep2024.csv', dtype={'ZIP Code':str})
    df = df.rename(columns={'CMS Certification Number (CCN)':'CCN','Provider Name':'provider_name','Provider Address':'provider_address','ZIP Code':'zip'})
    df = df[['CCN','provider_name','State','zip']]
    df['zip'] = df['zip'].astype(str)

    return df

provider_df = read_provider_df()


In [88]:
print(provider_df['zip'].str.len().value_counts())

zip
5    14817
Name: count, dtype: int64


In [89]:
def merge_survey_provider(survey_df, provider_df): 
    print('survey_df rows: ' + str(survey_df.shape[0]))
    print('provider_df rows: ' + str(provider_df.shape[0]))
    merge_df = pd.merge(survey_df, provider_df, how='left',on='CCN', indicator=True)
    print('survey_provider_df rows: ' + str(merge_df.shape[0]))
    print(merge_df['_merge'].value_counts())
    return merge_df

survey_provider_df = merge_survey_provider(survey_df, provider_df)

# CCN found for all providers

survey_df rows: 161369
provider_df rows: 14817
survey_provider_df rows: 161369
_merge
both          161369
left_only          0
right_only         0
Name: count, dtype: int64


In [90]:
def drop_fire_surveys(df): 
    df = df.loc[df['survey_type'].isin(['Fire Safety Standard','Fire Safety Complaint'])==0]
    print(df['survey_type'].value_counts())
    print(df.shape[0])
    return df 

drop_fire_surveys(survey_provider_df)

survey_type
Health Complaint     54456
Health Standard      44249
Infection Control    17878
Name: count, dtype: int64
116583


,CCN,survey_date,survey_type,survey_cycle,process_date,provider_name,State,zip,_merge
0,015009,2023-03-02,Health Complaint,2,2024-09-01,"BURNS NURSING HOME, INC.",AL,35653,both
1,015009,2023-03-02,Health Standard,1,2024-09-01,"BURNS NURSING HOME, INC.",AL,35653,both
4,015009,2019-08-21,Health Standard,2,2024-09-01,"BURNS NURSING HOME, INC.",AL,35653,both
6,015009,2018-08-01,Health Standard,3,2024-09-01,"BURNS NURSING HOME, INC.",AL,35653,both
7,015010,2023-08-30,Infection Control,2,2024-09-01,COOSA VALLEY HEALTHCARE CENTER,AL,35150,both
...,...,...,...,...,...,...,...,...,...
161361,745039,2023-06-20,Health Standard,1,2024-09-01,MIDTOWNE MEADOWS HEALTH AND REHAB,TX,76065,both
161362,745040,2023-10-13,Health Complaint,1,2024-09-01,THE SARAH ROBERTS FRENCH HOME,TX,78201,both
161363,745040,2023-10-13,Health Standard,1,2024-09-01,THE SARAH ROBERTS FRENCH HOME,TX,78201,both
161365,745040,2022-08-05,Health Standard,2,2024-09-01,THE SARAH ROBERTS FRENCH HOME,TX,78201,both


In [91]:
survey_provider_df['survey_date'] = pd.to_datetime(survey_provider_df['survey_date'])

survey_provider_df['year'] = survey_provider_df['survey_date'].dt.year
print(survey_provider_df['year'].value_counts())

year
2022    45039
2023    44661
2021    24515
2024    23841
2019    12655
2018     4843
2020     4650
2017     1080
2016       84
2015        1
Name: count, dtype: int64


In [92]:
def remove_surveys_after_date(df, cutoff_date): 
    # print(df.shape[0])
    df['cutoff_date'] = pd.to_datetime(cutoff_date)
    df_after_date = df.loc[df['survey_date']<df['cutoff_date']]
    # print(df_after_date['year'].value_counts())
    # print(df_after_date.shape[0])
    return df_after_date
    
# remove surveys after 09/30/2023. The FY2022 ends on 09/30/23, so we will only want to use survey information from before this to pair with state level surveyor position information. 
pre_oct2023_surveys_df = remove_surveys_after_date(survey_provider_df, '09-30-2023')

# dropped 34374 rows
# print(161369-126995)

In [93]:
print(len(pre_oct2023_surveys_df['CCN'].unique()))

14789


In [94]:
# calculate time between input date and last survey 
def identify_last_standard_survey(df): 
    """Identify the last standard survey for each nursing home"""
    df = df.loc[df['survey_type']=='Health Standard']
    df = df.sort_values(by=['CCN','survey_date'], ascending=False)
    print(df.shape[0]) # 
    df = df.drop_duplicates(subset=['CCN'], keep='first')
    print(df.shape[0]) # 14817, as it should be (total number of nursing homes) 
    return df 
    
last_standard_survey_df = identify_last_standard_survey(pre_oct2023_surveys_df)
    

35696
14788


In [95]:
def NH_months_since_last_standard_survey(df, cutoff_date): 
    df['cutoff_date'] = pd.to_datetime(cutoff_date)
    df['survey_date'] = pd.to_datetime(df['survey_date'])
    df['date_diff'] = df['cutoff_date'] - df['survey_date']
    df['months_since_survey'] = (df['cutoff_date'] - df['survey_date']).dt.days/30
    df['months_overdue'] = ((((df['cutoff_date'] - df['survey_date']).dt.days)/30)-15)
    df['overdue_ind'] = np.where(df['months_overdue']>1, 1, 0)
    print(df['months_overdue'].describe())
    print(df.head(10))
    df.to_csv('/Users/ghazalin/NH_inspections_static/OutputData/_093023_NH_months_since_last_standard_survey.csv')
    return df
    
NH_months_since_last_standard_survey = NH_months_since_last_standard_survey(last_standard_survey_df, '09-30-2023')

count    14788.000000
mean        -3.750586
std         10.355613
min        -14.966667
25%        -10.666667
50%         -6.333333
75%         -0.633333
max         48.833333
Name: months_overdue, dtype: float64
           CCN survey_date      survey_type  survey_cycle process_date  \
161365  745040  2022-08-05  Health Standard             2   2024-09-01   
161361  745039  2023-06-20  Health Standard             1   2024-09-01   
161359  745038  2023-05-19  Health Standard             1   2024-09-01   
161352  745022  2022-08-18  Health Standard             2   2024-09-01   
161342  745021  2023-09-26  Health Standard             1   2024-09-01   
161336  745019  2023-02-18  Health Standard             2   2024-09-01   
161315  745017  2023-08-17  Health Standard             1   2024-09-01   
161308  745007  2022-10-06  Health Standard             2   2024-09-01   
161289  745006  2023-02-23  Health Standard             2   2024-09-01   
161275  745005  2023-03-16  Health Standard    

In [96]:
print(NH_months_since_last_standard_survey.shape[0])

14788


In [97]:
def prop_overdue_surveys(df): 
    df['NH_count'] = 1
    df_groupby = df.groupby(['State']).agg({'overdue_ind':'sum','NH_count':'sum'}).reset_index()
    df_groupby['prop_overdue'] = df_groupby['overdue_ind']/df_groupby['NH_count']
    df_groupby = df_groupby.sort_values(by=['prop_overdue'])
    print(df_groupby)
    return df_groupby
    
prop_overdue_surveys_df = prop_overdue_surveys(NH_months_since_last_standard_survey)

   State  overdue_ind  NH_count  prop_overdue
0     AK            0        20      0.000000
3     AZ            0       142      0.000000
12    HI            0        42      0.000000
11    GU            0         1      0.000000
27    MT            0        59      0.000000
31    NH            0        74      0.000000
30    NE            0       184      0.000000
29    ND            0        75      0.000000
40    PR            0         6      0.000000
48    VT            0        34      0.000000
52    WY            0        35      0.000000
50    WI            0       324      0.000000
39    PA            0       668      0.000000
41    RI            0        74      0.000000
33    NM            0        67      0.000000
15    IL            1       679      0.001473
45    TX            2      1182      0.001692
19    LA            1       267      0.003745
2     AR            1       218      0.004587
43    SD            1        96      0.010417
34    NV            1        67   

In [98]:
print(len(prop_overdue_surveys_df['State'].value_counts()))

53


In [99]:
# read in surveyor position df 
surveyor_df = pd.read_csv('/Users/ghazalin/NH_inspections_static/data/state_NH_beds_surveyors.csv', header=1)
surveyor_df = surveyor_df[['State ', 'abbreviation', '#_NHs', '#_beds', 'surveyor_positions',
       'NH_per_position', 'beds_per_position']]
surveyor_df = surveyor_df.rename(columns={'State ':'state', 'abbreviation':'state_code'})
print(surveyor_df.columns)

Index(['state', 'state_code', '#_NHs', '#_beds', 'surveyor_positions',
       'NH_per_position', 'beds_per_position'],
      dtype='object')


In [100]:
merge_df = pd.merge(surveyor_df, prop_overdue_surveys_df, how='left', left_on='state_code', right_on='State')
merge_df['prop_overdue'] = (merge_df['prop_overdue']*100).astype(int)
print(merge_df.head(5))

        state state_code  #_NHs   #_beds  surveyor_positions  NH_per_position  \
0     Alabama         AL    231   48,851                  51              4.5   
1      Alaska         AK     20      830                  14              1.4   
2     Arizona         AZ    143   16,145                  35              4.1   
3    Arkansas         AR    221   25,700                  62              3.6   
4  California         CA  1,175  117,148                 654              1.8   

   beds_per_position State  overdue_ind  NH_count  prop_overdue  
0                958    AL          165       224            73  
1                 59    AK            0        20             0  
2                461    AZ            0       142             0  
3                415    AR            1       218             0  
4                179    CA          624      1163            53  


In [101]:
print(merge_df[['state','state_code']])

             state state_code
0          Alabama         AL
1           Alaska         AK
2          Arizona         AZ
3         Arkansas         AR
4       California         CA
5         Colorado         CO
6      Connecticut         CT
7         Delaware         DE
8          Florida         FL
9          Georgia         GA
10         Hawai’i         HI
11           Idaho         ID
12        Illinois         IL
13         Indiana         IN
14            Iowa         IA
15          Kansas         KS
16        Kentucky         KY
17       Louisiana         LA
18           Maine         ME
19        Maryland         MD
20   Massachusetts         MA
21        Michigan         MI
22       Minnesota         MN
23     Mississippi         MS
24        Missouri         MO
25         Montana         MT
26        Nebraska         NE
27          Nevada         NV
28   New Hampshire         NH
29      New Jersey         NJ
30      New Mexico         NM
31        New York         NY
32  North 

In [139]:
# create scatterplot of state proportion overdue surveys as of 09-30-2023 vs. state surveyor positions per NH bed

surveyor_beds_vs_prop_overdue_chart = alt.Chart(merge_df).mark_circle(color='#74372F').encode(alt.X('beds_per_position', title='Beds per Surveyor'), alt.Y('prop_overdue', title='Percent Overdue')).properties(title = alt.TitleParams(text='Percent Overdue by Beds per Surveyor', fontSize=15, dx=40))

surveyor_beds_vs_prop_overdue_chart = surveyor_beds_vs_prop_overdue_chart + surveyor_beds_vs_prop_overdue_chart.transform_regression('beds_per_position','prop_overdue').mark_line(color='#74372F')
surveyor_beds_vs_prop_overdue_chart = surveyor_beds_vs_prop_overdue_chart.properties(background='white')
display(surveyor_beds_vs_prop_overdue_chart)
surveyor_beds_vs_prop_overdue_chart.save('/Users/ghazalin/NH_inspections_static/images/surveyor_beds_vs_prop_overdue_scatter.svg')


alt.LayerChart(...)

In [140]:
NH_per_position_vs_prop_overdue_chart = alt.Chart(merge_df).mark_point(color='#74372F').encode(alt.X('NH_per_position',title='Facilities per Surveyor'),alt.Y('prop_overdue', title='Percent Overdue')
                                                                                              ).properties(title = alt.TitleParams(text='Percent Overdue by Facilities per Surveyor', fontSize=15, dx=20))

NH_per_position_vs_prop_overdue_chart = NH_per_position_vs_prop_overdue_chart + NH_per_position_vs_prop_overdue_chart.transform_regression('NH_per_position','prop_overdue').mark_line(color='#74372F')
NH_per_position_vs_prop_overdue_chart = NH_per_position_vs_prop_overdue_chart.properties(background='white')
display(NH_per_position_vs_prop_overdue_chart)
NH_per_position_vs_prop_overdue_chart.save('/Users/ghazalin/NH_inspections_static/images/NH_per_position_vs_prop_overdue_scatter.svg')

alt.LayerChart(...)

In [141]:
surveyor_count_vs_prop_overdue_chart = alt.Chart(merge_df).mark_point(color='#74372F').encode(alt.X('surveyor_positions',title= 'Budgeted Surveyor Positions'),alt.Y('prop_overdue', title='Percent Overdue')
                                                                                             ).properties(title = alt.TitleParams(text='Percent Overdue by Facilities per Surveyor', fontSize=15, dx=30))

surveyor_count_vs_prop_overdue_chart = surveyor_count_vs_prop_overdue_chart + surveyor_count_vs_prop_overdue_chart.transform_regression('surveyor_positions','prop_overdue').mark_line(color='#74372F')
surveyor_count_vs_prop_overdue_chart = surveyor_count_vs_prop_overdue_chart.properties(background='white')
display(surveyor_count_vs_prop_overdue_chart)
surveyor_count_vs_prop_overdue_chart.save('/Users/ghazalin/NH_inspections_static/images/surveyor_count_vs_prop_overdue_scatter.svg')

alt.LayerChart(...)